In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn.impute import KNNImputer

In [9]:
# load data: is "5_has_wettercode" the latest version?

df = pd.read_csv("/workspaces/bakery_sales_prediction/sourcedata/cleaned_data/6.3_weather_class.csv")

print("df:", "\n", df.head(), "\n")


df_wetter: 
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-01         6.0     17.8375                 15.0        20.0   
1  2013-07-01         6.0     17.8375                 15.0        20.0   
2  2013-07-01         6.0     17.8375                 15.0        20.0   
3  2013-07-01         6.0     17.8375                 15.0        20.0   
4  2013-07-01         6.0     17.8375                 15.0        20.0   

   KielerWoche  Warengruppe      Umsatz  Feiertage  Ferientage  
0          0.0          1.0  148.828353        0.0         1.0  
1          0.0          2.0  535.856285        0.0         1.0  
2          0.0          3.0  201.198426        0.0         1.0  
3          0.0          4.0   65.890169        0.0         1.0  
4          0.0          5.0  317.475875        0.0         1.0   

df_no_wetter: 
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  \
0  2013-07-02         3.0     17.3125                 10.0  

## 4. Saisonale Abweichungen:

In [22]:
import pandas as pd

# Assuming weather_data is your DataFrame and it is already loaded

# Step 1: Calculate the 7-day rolling average for temperature
df['temp_7d_avg'] = df['Temperatur'].rolling(window=7).mean()

# Step 2: Fill NA values created by rolling average
df['temp_7d_avg'].fillna(method='bfill', inplace=True)

# Step 3: Calculate the difference for the next day
df['temp_diff_next_day'] = df['Temperatur'].shift(-1) - df['temp_7d_avg']

# Step 4: Create 'warmer than week before' and 'colder than week before' columns
df['warmer_than_week_before'] = df['temp_diff_next_day'].apply(lambda x: x if x > 0 else 0)
df['colder_than_week_before'] = df['temp_diff_next_day'].apply(lambda x: -x if x < 0 else 0)

# Display the first few rows to check the results
print(df[['Temperatur', 'temp_7d_avg', 'temp_diff_next_day', 'warmer_than_week_before', 'colder_than_week_before']].head(15))
df.describe()


    Temperatur  temp_7d_avg  temp_diff_next_day  warmer_than_week_before  \
0      17.8375    18.762500           -0.925000                 0.000000   
1      17.8375    18.762500           -0.925000                 0.000000   
2      17.8375    18.762500           -0.925000                 0.000000   
3      17.8375    18.762500           -0.925000                 0.000000   
4      17.8375    18.762500            2.312500                 2.312500   
5      21.0750    18.762500            2.312500                 2.312500   
6      21.0750    18.762500            2.312500                 2.312500   
7      21.0750    19.225000            1.850000                 1.850000   
8      21.0750    19.687500            1.387500                 1.387500   
9      21.0750    20.150000           -1.300000                 0.000000   
10     18.8500    20.294643           -1.444643                 0.000000   
11     18.8500    20.439286           -1.589286                 0.000000   
12     18.85

/tmp/ipykernel_23735/483723604.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_wetter_seasons['temp_7d_avg'].fillna(method='bfill', inplace=True)
/tmp/ipykernel_23735/483723604.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_wetter_seasons['temp_7d_avg'].fillna(method='bfill', inplace=True)


,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Warengruppe,Umsatz,Feiertage,Ferientage,...,year,month,week,weekday,day_month,season,temp_7d_avg,temp_diff_next_day,warmer_than_week_before,colder_than_week_before
count,6999,6999.000000,6999.00000,6999.000000,6999.000000,6999.000000,6999.000000,6999.000000,6999.000000,6999.000000,...,6999.00000,6999.000000,6999.0,6999.000000,6999.000000,6999.000000,6999.000000,6998.000000,6999.000000,6999.000000
mean,2016-03-11 06:31:19.468495616,5.308473,10.72907,11.175025,36.894271,0.023575,3.100157,200.378479,0.019431,0.152450,...,2015.68767,6.581226,26.814831,3.017574,15.743678,2.608230,10.725512,0.005019,0.694784,0.689766
min,2013-07-01 00:00:00,0.000000,-8.47500,3.000000,0.000000,0.000000,1.000000,7.051201,0.000000,0.000000,...,2013.00000,1.000000,1.0,0.000000,1.000000,1.000000,-7.528571,-11.632143,0.000000,0.000000
25%,2014-12-17 12:00:00,4.000000,5.50000,8.000000,10.000000,0.000000,2.000000,94.626822,0.000000,0.000000,...,2014.00000,3.000000,12.0,1.000000,8.000000,2.000000,5.553571,-0.982143,0.000000,0.000000
50%,2016-03-15 00:00:00,6.000000,10.00000,10.000000,28.000000,0.000000,3.000000,155.268163,0.000000,0.000000,...,2016.00000,7.000000,27.0,3.000000,16.000000,3.000000,10.035714,-0.042857,0.000000,0.042857
75%,2017-06-18 00:00:00,7.000000,16.38750,14.000000,61.000000,0.000000,4.000000,271.448535,0.000000,0.000000,...,2017.00000,10.000000,42.0,5.000000,23.000000,4.000000,16.388393,1.000000,1.000000,0.982143
max,2018-07-30 00:00:00,8.000000,28.87500,35.000000,95.000000,1.000000,6.000000,1879.461831,1.000000,1.000000,...,2018.00000,12.000000,53.0,6.000000,31.000000,4.000000,28.746429,12.250000,12.250000,11.632143
std,NaN,2.392562,6.84571,4.334977,27.076717,0.151731,1.497201,140.549060,0.138045,0.359483,...,1.52465,3.670393,16.015185,1.993661,8.735942,1.139948,6.725947,1.910794,1.172045,1.148175


In [23]:
df_w = df

In [ ]:
# check what exactly this means and does! (Suggestion by ChatGPT)
# Create rolling averages for temperature, precipitation and and wind speed
# weather_data['temp_7d_avg'] = weather_data['Temperatur'].rolling(window=7).mean()
# weather_data['wind_7d_avg'] = weather_data['Windgeschwindigkeit'].rolling(window=7).mean()
# weather_data['prec_7d_avg'] = weather_data['Niederschlag'].rolling(window=7).mean()

# Create lag features
# weather_data['temp_lag_1'] = weather_data['Temperatur'].shift(1)
# weather_data['wind_lag_1'] = weather_data['Windgeschwindigkeit'].shift(1)
# weather_data['prec_lag_1'] = weather_data['Niederschlag'].shift(1)

# Fill NA values created by rolling and lag features
# weather_data.fillna(method='bfill', inplace=True)

# Normalize the weather features
# from sklearn.preprocessing import StandardScaler

# weather_features = ['Temperatur', 'Windgeschwindigkeit', 'Niederschlag', 'temp_7d_avg', 'wind_7d_avg', 'temp_lag_1', 'wind_lag_1', 'prec_7d_avg', 'prec_lag_1']
# scaler = StandardScaler()
# weather_data[weather_features] = scaler.fit_transform(weather_data[weather_features])
# print(weather_data.head())


In [ ]:
# Open ToDos:
# Create categories for temperature depending on season
# or calculate the deviation from monthly temperature
# or define warm / cold day depending on the season (e.g. in winter 10°C is a warm day, in summer that would be a cold day)

In [ ]:
# Define thresholds for extreme weather conditions
VERY_HOT_THRESHOLD = 25  # Example: temperature > 25 degrees Celsius
VERY_COLD_THRESHOLD = 0  # Example: temperature < 0 degrees Celsius
VERY_RAINY_THRESHOLD = 10 # Example: precipitation > 10 mm
HIGH_WIND_THRESHOLD = 15  # Example: wind speed > 15 m/s
SUNNY_DAY_THRESHOLD = 2  # Example: cloudiness <= 2 - how is the parameter defined?

# Create binary features for extreme weather conditions
weather_data['very_hot'] = (weather_data['Temperatur'] > VERY_HOT_THRESHOLD).astype(int)
weather_data['very_cold'] = (weather_data['Temperatur'] < VERY_COLD_THRESHOLD).astype(int)
weather_data['very_rainy'] =  (weather_data['Niederschlag'] > VERY_RAINY_THRESHOLD).astype(int)
weather_data['high_wind'] = (weather_data['Windgeschwindigkeit'] > HIGH_WIND_THRESHOLD).astype(int)
weather_data['sunny_days'] = (weather_data['Bewoelkung'] <= SUNNY_DAY_THRESHOLD).astype(int)

# Display the first few rows to check all features
print(weather_data)


In [ ]:
# Calculate "feels like" temperature
weather_data["Windgeschwindigkeit_km_h"] = weather_data["Windgeschwindigkeit"] * 3.6
weather_data["feels_like_temperature"] = 13.12 + 0.6215 * weather_data["Temperatur"] - 11.37 * weather_data["Windgeschwindigkeit_km_h"]**0.16 + 0.3965*weather_data["Temperatur"]*weather_data["Windgeschwindigkeit_km_h"]**0.16
print(weather_data.head())

# export file again
# weather_data.to_csv("Files/weather_data.csv", index = False)


## Ergebnis speichern:

In [24]:
 # Save the updated data to new CSV files
df_w.to_csv('../sourcedata/cleaned_data/6.4_temperature_deviation-2.csv', sep=',', index=False)